# Привет! Тут мы анализируем русскоязычные твиты :)

#### Датасет:
https://drive.google.com/drive/folders/0BxlA8wH3PTUfV1F1UTBwVTJPd3c    
#### Необходимые библиотеки
numpy, pandas, scikit-learn, gensim, tensorflow-gpu, keras    
#### Задания:
1. Успешно запустить имеющийся ноутбук
2. Запустить этот же ноутбук на данных ___ttk___
3. Реализовать свёрточную нейронную сеть для анализа твитов:
    - По аналогии с этой статьёй https://www.aclweb.org/anthology/D14-1181.
    - На вход нейросети будут поступать настаканные друг на друга эмбеддинги поочерёдно каждого слова в тексте.
    - Реализацию архитектуры статьи делайте по аналогии с примерами обработки картинок с помощью Sequential, Conv2D, MaxPool, Dense, Dropout из keras.
    - Важные моменты:    
        \> Результирующий X будет 3D матрицей (кол-во сэмплов \* максимальная длина текста \* длину вектора эмбеддинга)    
        \> padding входных матриц (твиты - переменного размера, а нужно фиксированного, поэтому нужно матрицы добивать нулями или обрезать)    
        \> Чтобы адаптировать примеры с картинками к примерам с текстами, нужно аккуратно поменять размер рецептивного поля у Conv2D    
        \> Для keras нужно передавать метки класса в sparce-формате (П: не y=\[0,1,2\], a y=\[\[1,0,0\], \[0,1,0\], \[0,0,1\]\])
        \> Для улучшения результата можно вектора слов умножать на их Tf-Idf score, можно нормализовать весь X, добавлять Dropout в модель

Если будут вопросы - обращайтесь :)

## I. Импортим нужные библиотеки

In [1]:
import xmltodict
import re

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

## II. Определяем функции с парсингом входных данных

In [2]:
companies = {}
def get_sample_text(sample):
    assert sample['column'][3]['@name'] == 'text'
    return sample['column'][3]['#text']


def get_sample_answers_bank(sample):
    answers = {}
    for i in range(4, 12):
        companies[sample['column'][i]['@name']] = i
        answers[sample['column'][i]['@name']] = None if sample['column'][i]['#text'] == 'NULL'\
            else int(sample['column'][i]['#text'])
    return answers

def get_sample_answers_tkk(sample):
    answers = {}
    for i in range(4, 11):
        companies[sample['column'][i]['@name']] = i
        answers[sample['column'][i]['@name']] = None if sample['column'][i]['#text'] == 'NULL'\
            else int(sample['column'][i]['#text'])
    return answers

def get_sample_id(sample):
    assert sample['column'][0]['@name'] == 'id'
    return int(sample['column'][0]['#text'])


def get_data(filename):
    df = pd.DataFrame()
    with open(filename, "r", encoding='utf-8') as f:
        d = xmltodict.parse(f.read(), process_namespaces=True)
        clean_samples = []
        for sample in d['pma_xml_export']['database']['table']:
            sample_id = get_sample_id(sample)
            text = get_sample_text(sample)
            answers = get_sample_answers_bank(sample)
            for company, answer in answers.items():
                if answer is not None:
                    clean_samples.append((sample_id, text, company, answer))
        df['text'] = [sample[1] for sample in clean_samples]
        df['answer'] = [sample[3] for sample in clean_samples]
        df['company'] = [sample[2] for sample in clean_samples]
        df['sample_id'] = [sample[0] for sample in clean_samples]
    return df

## III. Обрабатываем наши данные

In [3]:
train_filename = "/home/ds/DataScience/Datasets/SentiRuEval_2016/bank_train_2016.xml"
test_filename = "/home/ds/DataScience/Datasets/SentiRuEval_2016/banks_test_etalon.xml"

train = get_data(train_filename)
test = get_data(test_filename)

## IV. Заменяем все ссылки и юзернеймы на url и user соответственно

In [4]:
url_replacement = lambda x: re.sub(r'(?:http[^\s]+)($|\s)', r'url\1', x)
user_replacement = lambda x: re.sub(r'(?:@[^\s]+)($|\s)', r'user\1', x)

train['text'] = train['text'].apply(url_replacement)
train['text'] = train['text'].apply(user_replacement)

test['text'] = test['text'].apply(url_replacement)
test['text'] = test['text'].apply(user_replacement)

In [5]:
train.head()

,text,answer,company,sample_id
0,url Взять кредит тюмень альфа банк,0,alfabank,1
1,Мнение о кредитной карте втб 24 url,0,vtb,2
2,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,0,raiffeisen,3
3,Современное состояние кредитного поведения в р...,0,sberbank,4
4,user user Главное чтоб банки СБЕР и ВТБ!!!,1,sberbank,5


In [6]:
train['answer'].value_counts()

 0    7158
-1    2807
 1     760
Name: answer, dtype: int64

In [7]:
test.head()

,text,answer,company,sample_id
0,#Автокредит в россельхозбанк в череповце,0,rshb,1
1,RT user url #Кредитный калькулятор россельхозб...,0,rshb,2
2,#Автокредит в россельхозбанк 2012 url,0,rshb,3
3,RT user #Кредитные карты россельхозбанк url,0,rshb,4
4,RT user #Кредиты в россельхозбанке ижевск url,0,rshb,5


## V. Преобразование нашей обучающей выборки по tf-idf

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
X_train = vectorizer.fit_transform(train['text'])

In [10]:
X_train.shape

(10725, 11703)

## VI. Преобразование нашей тестовой выборки

In [11]:
X_test = vectorizer.transform(test['text'])

In [12]:
X_test.shape

(3418, 11703)

## VII. Обучение логистической регрессии

In [13]:
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', class_weight='balanced')

In [14]:
lr.fit(X_train, train['answer'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=None, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)

In [15]:
y_pred = lr.predict(X_test)

## VIII. Смотрим на результаты
Считается F1 мера с микро и макро усреднением по негативному и положительному классу как в соревновании

In [16]:
f1_score(test['answer'], y_pred, average='micro', labels=[-1,1])

0.49823460180462925

In [17]:
f1_score(test['answer'], y_pred, average='macro', labels=[-1,1])

0.4698891720341647

.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
.    
